In [21]:
from pymongo import MongoClient

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

import time

import re
from urllib.parse import urlparse

from bs4 import BeautifulSoup

from IPython.display import clear_output


In [2]:

client = MongoClient('mongodb://localhost:27017/')
db = client['Lowerated']
collection = db['RottenTomatoes']

CollectionList = list(collection.find({}))
CollectionList[0]


{'_id': 0,
 'url': 'https://www.rottentomatoes.com/m/shawshank_redemption',
 'audienceScore': '98%',
 'criticsScore': '89%',
 'Critic Reviews': [{'ReviewID': 96,
   'Critic': 'The Shawshank Redemption: Special Edition\nEvery movie trivia buff worth his or her salt can rattle off the titles of at least half a dozen films that tanked when they first hit theaters, only to be recognized later as all-time favorites. Some fall under the "cult" banner — Office Space, The Princess Bride (fans as rabid as Buttercup and Westley\'s definitely qualify for cult standing) — but others are movies that have become synonymous with "classic": It\'s a Wonderful Life and Casablanca are two prime examples ... and so is The Shawshank Redemption (1994). Director Frank Darabont\'s moving period prison drama (based on Stephen King\'s short story "Rita Hayworth and Shawshank Redemption") earned only $28 million at the box office, yet 10 years after its release, it regularly tops popularity polls. Perhaps movieg

In [22]:

driver = webdriver.Chrome()
driver.maximize_window()


In [23]:

for data in CollectionList:
    
    movie_id = data['_id']
    if movie_id > 5:
        url = data['url']
        driver.get(url)
        time.sleep(1)

        driver.execute_script("window.scrollBy(0, 1100);")

        try:
            TotalAudience = driver.find_element(By.XPATH, '//*[@id="modules-wrap"]/section[2]/div[1]/rt-link')
        except Exception as e:
            print('No Audiance Review found')
            TotalAudience = None
                        
                        
        if TotalAudience:
            Totaltxt = TotalAudience.text
            
            match = re.search(r'\((\d+)\+', Totaltxt)
            if match:
                TotalAudienceReviews = int( match.group(1) )  
            else:
                
                match = re.search(r'\((\d+)\)', Totaltxt)
                if match:
                    TotalAudienceReviews = int(match.group(1))  
                else:
                    TotalAudienceReviews = None
                    print('Number not found Regex')    
        
        
        
            if TotalAudienceReviews:
                parsed_url = urlparse(url)
                movie_name = parsed_url.path.split('/m/')[-1]
            
                driver.get(f'https://www.rottentomatoes.com/m/{movie_name}/reviews?type=user')
                time.sleep(1)

            CurrentMovieAllReviews = []
            TotalAudienceReviews -= 1
            for ReviewNum in range(1, TotalAudienceReviews):
                
                # --------------------- Press Load More after every 20 Reviews ---------------------
                if ReviewNum % 20 == 0:
                    print(f"Now more reviews are available and uploaded Total ReviewNum completed are: {ReviewNum}", end= " ")
                    MoreReviews = driver.find_element(By.XPATH, '//*[@id="reviews"]/div[2]/rt-button')
                    MoreReviews.click()                         
                
                        
                try:                                                                                                    
                    ReviewElement = driver.find_element(By.XPATH, f'//*[@id="reviews"]/div[1]/div[{ReviewNum}]/div[2]/drawer-more/p')
                    Reviewtxt = ReviewElement.text
                    
                    print(Reviewtxt[:10])
                    
                except NoSuchElementException:
                    print('EXCEPTION No Element found')
                    time.sleep(360)

                driver.execute_script("window.scrollBy(0, 150);")
                time.sleep(1)
                        
                CurrentReview ={
                    'ReviewID':	ReviewNum,
                    'Critic': Reviewtxt,
                }
                        
                CurrentMovieAllReviews.append(CurrentReview)
                
            movie_id = data['_id']
            CurrentMovie = {
                    "_id": movie_id,
                    "Reviews": CurrentMovieAllReviews
                }
                    
            print(f'Total reviews: {ReviewNum} collected for movie: {movie_name}, so Total movies done: {movie_id}')    
                    
            collection.update_one(
                {'_id': CurrentMovie['_id']},
                {'$set': {'Audience Reviews': CurrentMovie['Reviews']}},
                upsert=True
            )
            
            clear_output(wait=True)
        

Peter Jack
Not much n
One of the
The Best F
Classics! 
A perfect 
This movie
The uplift
My favouri
The greate
Truly perf
This refle
It is sad 
Classic ta
Very rarel
Perfect en
I hesitate
The Lord o
Loved it w
Now more reviews are available and uploaded Total ReviewNum completed are: 20 Definitely
I love the
Some of th
Peak Cinem
Still the 
The best m
it maybe o
Brought my
Un espectá
The Return
8.7/10 Tha
Legendary!
The near p
Uma obra p
10 Stars I
Best of th
best of th
the greate
Previously
Incredible
Now more reviews are available and uploaded Total ReviewNum completed are: 40 A true cla
Simply the
Great movi
Very possi
Excellent.
The visual
Great movi
Such an am
One of the
A perfect 
This is th
The defini
I think it
Fantastic 
Unlike the
An absolut
Adoro a sa
The perfec
Flawlessly
One of the
Now more reviews are available and uploaded Total ReviewNum completed are: 60 10/10 one 
One of bes
The only t
The best e
Do you rea
The Best. 
The last m
Absolutely
As Gandalf
Aamzing Ke
After 2

In [14]:
ReviewNum

235

In [12]:

Totaltxt


'View All (1000+)\naudience reviews'